In [1]:
import findspark
findspark.init()

In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from surprise import SVD, SVDpp, NMF, Dataset, Reader
from surprise.model_selection import cross_validate
import pandas as pd

In [4]:
spark = SparkSession.builder.appName("CollaborativeFiltering").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/19 02:20:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
hotel_comments_df = pd.read_csv("data/hotel_comments_cleaned.csv")
hotel_info_df = pd.read_csv("data/hotel_info_cleaned.csv")

In [6]:
# Tiền xử lý dữ liệu cho mô hình PySpark ALS
als_data = hotel_comments_df[['Hotel ID', 'Reviewer ID', 'Score']]
als_data['Hotel ID'] = als_data['Hotel ID'].apply(lambda x: int(x.split('_')[1]))
als_data['Reviewer ID'] = als_data['Reviewer ID'].apply(lambda x: int(x.split('_')[2]))

/var/folders/py/l7rngj8j6sl2flg1m973yfgh0000gn/T/ipykernel_7275/1463577058.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  als_data['Hotel ID'] = als_data['Hotel ID'].apply(lambda x: int(x.split('_')[1]))
/var/folders/py/l7rngj8j6sl2flg1m973yfgh0000gn/T/ipykernel_7275/1463577058.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  als_data['Reviewer ID'] = als_data['Reviewer ID'].apply(lambda x: int(x.split('_')[2]))


In [7]:
# Chuyển đổi DataFrame sang Spark DataFrame
spark_df = spark.createDataFrame(als_data)

In [8]:
reviewers = spark_df.select("Reviewer ID").distinct().count()
hotels = spark_df.select("Hotel ID").distinct().count()
numerator = spark_df.count()
display(numerator, reviewers, hotels)

25892

250

30

In [9]:
# Chia dữ liệu thành tập đào tạo và kiểm tra
(training, test) = spark_df.randomSplit([0.8, 0.2])

In [10]:
# Xây dựng mô hình ALS
als = ALS(maxIter=10, regParam=0.1, rank=25, userCol="Reviewer ID", itemCol="Hotel ID", ratingCol="Score", coldStartStrategy="drop", nonnegative=True)
model = als.fit(training)

24/08/19 02:20:52 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [11]:
# Đánh giá mô hình
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="Score", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

In [12]:
print(f"Root-mean-square error = {str(rmse)}")

Root-mean-square error = 0.9801655539339178


In [13]:
predictions.show(5)

+--------+-----------+-----+----------+
|Hotel ID|Reviewer ID|Score|prediction|
+--------+-----------+-----+----------+
|      23|        148| 10.0|  9.462127|
|       1|         31| 10.0|  9.078973|
|      11|         31| 10.0|  9.140502|
|      13|         31|  7.2|  9.205994|
|      15|         31| 10.0|  8.665439|
+--------+-----------+-----+----------+
only showing top 5 rows



In [14]:
# Tải dữ liệu cho Surprise
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(hotel_comments_df[['Reviewer ID', 'Hotel ID', 'Score']], reader)

In [15]:
# Huấn luyện và đánh giá mô hình SVD
svd = SVD()
svd_results = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9358  0.9538  0.9626  0.9396  0.9376  0.9459  0.0105  
MAE (testset)     0.7499  0.7655  0.7703  0.7554  0.7531  0.7588  0.0077  
Fit time          0.28    0.28    0.39    0.27    0.26    0.30    0.04    
Test time         0.26    0.02    0.03    0.02    0.02    0.07    0.09    


In [16]:
# Huấn luyện và đánh giá mô hình SVD++
svdpp = SVDpp()
svdpp_results = cross_validate(svdpp, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9321  0.9440  0.9523  0.9509  0.9610  0.9481  0.0096  
MAE (testset)     0.7598  0.7627  0.7735  0.7672  0.7719  0.7670  0.0052  
Fit time          0.21    0.20    0.21    0.20    0.20    0.20    0.01    
Test time         0.02    0.08    0.02    0.02    0.02    0.03    0.03    


In [17]:
# Huấn luyện và đánh giá mô hình NMF
nmf = NMF()
nmf_results = cross_validate(nmf, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9888  0.9679  0.9697  0.9588  0.9978  0.9766  0.0144  
MAE (testset)     0.8008  0.7870  0.7924  0.7840  0.8070  0.7942  0.0086  
Fit time          1.41    1.28    1.35    1.34    1.34    1.34    0.04    
Test time         0.02    0.02    0.02    0.02    0.02    0.02    0.00    


- SVD có RMSE trung bình là khoảng 0.9456 và MAE trung bình là 0.7589. Thời gian fit và test khá nhanh, chỉ khoảng 0.22s và 0.02s tương ứng.
- SVD++ cho thấy RMSE trung bình là khoảng 0.9477 và MAE trung bình là 0.7662. Thời gian fit và test là 0.16s và 0.01s tương ứng.
- NMF cho kết quả RMSE trung bình là 0.9766 và MAE trung bình là 0.7942. Thời gian fit của mô hình này lâu nhất là 0.86s, nhưng thời gian test tương tự như hai mô hình còn lại.

=> Mô hình SVD là một lựa chọn phù hợp cho bài toán này, nhờ vào sự cân bằng giữa độ chính xác và thời gian huấn luyện.

In [18]:
# Xử dụng mô hình SVD để xây dựng hàm đề xuất

# Chuẩn bị dữ liệu
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(hotel_comments_df[['Reviewer ID', 'Hotel ID', 'Score']], reader)
trainset = data.build_full_trainset()

# Huấn luyện mô hình SVD
svd_model = SVD()
svd_model.fit(trainset)

def recommend_hotels(reviewer_id, num_recommendations=5):
    # Lấy danh sách tất cả các khách sạn
    all_hotels = hotel_info_df['Hotel_ID'].unique()
    
    # Lấy danh sách các khách sạn đã đánh giá bởi người dùng này
    rated_hotels = hotel_comments_df[hotel_comments_df['Reviewer ID'] == reviewer_id]['Hotel ID'].unique()
    
    # Lọc ra các khách sạn chưa được đánh giá bởi người dùng này
    unrated_hotels = [hotel for hotel in all_hotels if hotel not in rated_hotels]
    
    # Dự đoán điểm cho các khách sạn chưa được đánh giá
    predictions = [svd_model.predict(reviewer_id, hotel).est for hotel in unrated_hotels]
    
    # Tạo DataFrame chứa kết quả và sắp xếp theo điểm dự đoán
    recommendations = pd.DataFrame({
        'Hotel_ID': unrated_hotels,
        'Predicted_Score': predictions
    })
    
    # Lấy top các khách sạn được đề xuất cao nhất
    top_recommendations = recommendations.sort_values(by='Predicted_Score', ascending=False).head(num_recommendations)
    
    # Trả về thông tin chi tiết của các khách sạn được đề xuất
    return hotel_info_df[hotel_info_df['Hotel_ID'].isin(top_recommendations['Hotel_ID'])][['Hotel_ID', 'Hotel_Name', 'Hotel_Rank', 'Hotel_Address', 'Total_Score']]

# Ví dụ sử dụng hàm
print(recommend_hotels(reviewer_id='148', num_recommendations=5))

    Hotel_ID                                       Hotel_Name      Hotel_Rank  \
32       2_3                            ELITE HOTEL NHA TRANG    4 sao trên 5   
40      2_11                            Maris Hotel Nha Trang    4 sao trên 5   
198    16_15          HANZ Muong Thanh Vien Trieu Condo Hotel  No information   
272    29_14                Moonlight Bay Panorama Ocean View  No information   
620     20_2  Khách Sạn MerPerle Beach (MerPerle Beach Hotel)  No information   

                                         Hotel_Address  Total_Score  
32   50 Đ. Củ Chi, Vĩnh Hải, ELite Hotel Nha Trang,...          9.7  
40   27 Trần Quang Khải, Phường Lộc Thọ, Thành phố ...          9.2  
198  5 Phạm Văn Đồng, Vĩnh Hải, Nha Trang, Khánh Hò...         10.0  
272  02 Nguyễn Thị Minh Khai, Lộc Thọ, Nha Trang, V...          9.1  
620  88A Tran Phu Street, Lộc Thọ, Nha Trang, Việt ...          9.2  


In [19]:
# spark.stop()
